Author: Samhaa El-Beltagy

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
!pip install tiktoken
!pip install chromadb

In [ ]:
import os
import openai
import sys
sys.path.append('../..')

from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install langchain-community langchain_chroma langchain_openai

In [ ]:
import os
import pandas as pd
import numpy as np
import gensim

from langchain.embeddings.openai import OpenAIEmbeddings

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.vectorstores import Chroma
from langchain import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter


In [ ]:
file_path = 'drive/My Drive/Colab Notebooks/Rag_Test/ACLing'

## General code for all models

In [ ]:
ARCD_file_path = 'drive/My Drive/Colab Notebooks/Rag_Test/ACLing/allData_mod2.csv'
df = pd.read_csv(ARCD_file_path)
df.head()

,Column1,id,title,context,question,answers,modified_question (GPT 3.5 Turbo),modified_question
0,0,6.220000e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,من هو حمزة بن عبد المطلب؟,{'text': array(['صحابي من صحابة رسول الإسلام م...,من هو حمزة بن عبد المطلب؟,من هو حمزة بن عبد المطلب؟
1,1,1.890000e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصفه رسول الله؟,"{'text': array(['وَخَيْرُ أَعْمَامِي'], dtype=...",بما وصف رسول الله حمزة بن عبد المطلب؟,بما وصف رسول الله حمزة بن عبد المطلب؟
2,2,6.630000e+11,حمزة بن عبد المطلب,حمزة بن عبد المطلب الهاشمي القرشي صحابي من صحا...,بما وصف رسول الله على ؟,"{'text': array(['«خَيْرُ إِخْوَتِي عَلِيٌّ،'],...",بما وصف رسول الله عليّ؟,بما وصف رسول الله عليّ؟
3,3,5.014659e+10,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,متى اسلم حمزة؟,{'text': array(['في السنة الثانية من بعثة النب...,متى أسلم حمزة؟,متى أسلم حمزة؟
4,4,2.600000e+11,حمزة بن عبد المطلب,أسلم حمزة في السنة الثانية من بعثة النبي محمد،...,و ماذا فعل فى غزوة بدر؟,{'text': array(['وقَتَلَ فيها شيبة بن ربيعة مب...,و ماذا فعل حمزة في غزوة بدر؟,و ماذا فعل حمزة في غزوة بدر؟


In [ ]:
documents = list(set(df['context']))
len(documents)


460

In [ ]:
def calcRecValues(df, k,  retriever):
  recall_Matrix = np.zeros((len(df), k))
  for i in range(len(df)):
    x = retriever.get_relevant_documents(df['question'].iloc[i])
    c = df['context'].iloc[i]
    for j in range(len(x)):
      if c == x[j].page_content:
        for l in range(j,k):
            recall_Matrix[i][l] = 1
  return recall_Matrix

In [ ]:
def calcRecValues2(df, k,  retriever):
  recall_Matrix = np.zeros((len(df), k))
  for i in range(len(df)):
    x = retriever.get_relevant_documents(df['modified_question'].iloc[i])
    c = df['context'].iloc[i]
    for j in range(len(x)):
      if c == x[j].page_content:
        for l in range(j,k):
            recall_Matrix[i][l] = 1
  return recall_Matrix

In [ ]:
def calcRecValues3(df, k,  retriever):
  recall_Matrix = np.zeros((len(df), k))
  for i in range(len(df)):
    x = retriever.get_relevant_documents(df['modified_question (GPT 3.5 Turbo)'].iloc[i])
    c = df['context'].iloc[i]
    for j in range(len(x)):
      if c == x[j].page_content:
        for l in range(j,k):
            recall_Matrix[i][l] = 1
  return recall_Matrix

In [ ]:
def getRecallAtK(recall_Matrix, k):
  column_sum = recall_Matrix[:, k-1].sum()
  return column_sum/len(recall_Matrix)

In [ ]:
def calculate_MRR(arr):
  total = len(arr)
  MRR = 0
  for i in range(total):
    for j in range(len(arr[i])):
      if arr[i][j] == 1:
        MRR += 1/(j+1)
        break
  return MRR/total


In [ ]:
def printRecallValues(recall_Matrix):
  print("recall @1:", getRecallAtK(recall_Matrix, 1))
  print("recall @3:", getRecallAtK(recall_Matrix, 3))
  print("recall @5:", getRecallAtK(recall_Matrix, 5))
  print(getRecallAtK(recall_Matrix, 1), getRecallAtK(recall_Matrix, 3), getRecallAtK(recall_Matrix, 5))

## Calc Average Word and Char Length

In [ ]:
def calculate_average_length_in_words(df, qfieldName):

    total_words = df[qfieldName].str.split().apply(len).sum()

    # Count the number of questions
    f_count = df[qfieldName].count()

    # Calculate the average length in words
    if f_count == 0:
        return 0
    average_length_in_words = total_words / f_count

    return average_length_in_words

def calculate_average_length_in_chars(df, qfieldName):

    total_length =df[qfieldName].str.len().sum()
    t_count = df[qfieldName].count()
    # Calculate the average length
    if t_count == 0:
        return 0
    average_length = total_length / t_count
    return average_length

In [ ]:
print(calculate_average_length_in_words(df, 'context'))
print(calculate_average_length_in_words(df, 'question'))
print(calculate_average_length_in_chars(df, 'context'))
print(calculate_average_length_in_chars(df, 'question'))

96.59139784946237
6.230107526881721
575.1096774193549
33.2831541218638


## OpenAI


In [ ]:
api_key_path = '/content/drive/My Drive/Colab Notebooks/Rag_Test/openApiKey.txt'

with open(api_key_path, 'r') as file:
    openai_api_key = file.read().strip()

# Set the environment variable
os.environ['OPENAI_API_KEY'] = openai_api_key

# Verify that the environment variable is set correctly

openai.api_key  = os.environ['OPENAI_API_KEY']


In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
persist_directory = file_path+'/chroma/openAI/'

In [ ]:
newDb = True  #change to False if you have already generated a vector DB in a prevoius run

In [ ]:
if newDb:
  !rm -rf persist_directory # remove old database files if any
  #vectorstore_openai = Chroma.from_documents(documents=documents, embeddings=embedding, persist_directory=persist_directory)
  vectorstore_openai=  Chroma.from_texts(texts=documents, embedding=embedding, persist_directory=persist_directory)
  vectorstore_openai.persist()
  newDb = False
else:
  vectorstore_openai = Chroma(persist_directory=persist_directory, embedding_function=embedding)
  print(vectorstore_openai._collection.count())
retriever_openai = vectorstore_openai.as_retriever(search_type="similarity",search_kwargs={'k': 5})

460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_openai)
printRecallValues(recall_Matrix)
op_MRR = calculate_MRR(recall_Matrix)
print(op_MRR)

recall @1: 0.5146953405017921
recall @3: 0.7362007168458782
recall @5: 0.7899641577060932
0.5146953405017921 0.7362007168458782 0.7899641577060932
0.6266666666666677


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_openai)
printRecallValues(recall_Matrix)
op_MRR2 = calculate_MRR(recall_Matrix)
print(op_MRR2)

recall @1: 0.5440860215053763
recall @3: 0.7770609318996415
recall @5: 0.8308243727598567
0.5440860215053763 0.7770609318996415 0.8308243727598567
0.6618399044205507


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_openai)
printRecallValues(recall_Matrix)
op_MRR3 = calculate_MRR(recall_Matrix)
print(op_MRR3)

recall @1: 0.5218637992831541
recall @3: 0.7670250896057348
recall @5: 0.817921146953405
0.5218637992831541 0.7670250896057348 0.817921146953405
0.6444205495818407


## AraVec

In [ ]:
model_path = 'drive/My Drive/aravec/wikipedia_sg_300'
aravec_model = gensim.models.Word2Vec.load(model_path)

In [ ]:
def clean_str(text):
    search = ["أ","إ","آ","ة","_","-","/",".","،"," و "," يا ",'"',"ـ","'","ى","\\",'\n', '\t','&quot;','?','؟','!']
    replace = ["ا","ا","ا","ه"," "," ","","",""," و"," يا","","","","ي","",' ', ' ',' ',' ? ',' ؟ ',' ! ']

    #remove tashkeel
    p_tashkeel = re.compile(r'[\u0617-\u061A\u064B-\u0652]')
    text = re.sub(p_tashkeel,"", text)

    #remove longation
    p_longation = re.compile(r'(.)\1+')
    subst = r"\1\1"
    text = re.sub(p_longation, subst, text)

    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    for i in range(0, len(search)):
        text = text.replace(search[i], replace[i])

    #trim
    text = text.strip()

    return text

In [ ]:
# Define the embedding function using AraVec
def aravec_embedding(text):
    text = clean_str(text)
    words = text.split()
    embeddings = [aravec_model.wv[word] for word in words if word in aravec_model.wv.index_to_key]
    if embeddings:
        mean_embedding = np.mean(embeddings, axis=0)
        return mean_embedding.tolist()
    else:
        return np.zeros(aravec_model.vector_size).tolist()

# Create the embedding function for Chroma
class AraVecEmbeddings:
    def embed_documents(self, texts):
        return [aravec_embedding(text) for text in texts]

    def embed_query(self, text):
        return aravec_embedding(text)

# Use AraVec embeddings
aravec_embeddings = AraVecEmbeddings()

In [ ]:
import re

In [ ]:
persist_directory = file_path+'/chroma/AraVec/SG/'
newAravecDb1 = False
if newAravecDb1:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_aravec_sg=  Chroma.from_texts(texts=documents, embedding=aravec_embeddings, persist_directory=persist_directory)
  vectorstore_aravec_sg.persist()
else:
  vectorstore_aravec_sg = Chroma(persist_directory=persist_directory, embedding_function=aravec_embeddings)
  print(vectorstore_aravec_sg._collection.count())
retriever_aravec = vectorstore_aravec_sg.as_retriever(search_type="similarity", search_kwargs={'k': 5})


/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_aravec)
printRecallValues(recall_Matrix)
aravec_MRR = calculate_MRR(recall_Matrix)
print(aravec_MRR)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


recall @1: 0.36129032258064514
recall @3: 0.5627240143369175
recall @5: 0.6236559139784946
0.36129032258064514 0.5627240143369175 0.6236559139784946
0.4663082437275987


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_aravec)
printRecallValues(recall_Matrix)

recall @1: 0.36702508960573477
recall @3: 0.5749103942652329
recall @5: 0.6508960573476702
0.36702508960573477 0.5749103942652329 0.6508960573476702


In [ ]:
aravec_MRR2 = calculate_MRR(recall_Matrix)
print(aravec_MRR2)

0.477861409796894


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_aravec)
printRecallValues(recall_Matrix)
aravec_MRR3 = calculate_MRR(recall_Matrix)
print(aravec_MRR3)

recall @1: 0.36129032258064514
recall @3: 0.5627240143369175
recall @5: 0.6236559139784946
0.36129032258064514 0.5627240143369175 0.6236559139784946
0.4663082437275987


In [ ]:
print(vectorstore_aravec_cbow._collection.count())


AttributeError: 'str' object has no attribute '_collection'

In [ ]:
model_path = 'drive/My Drive/aravec/wikipedia_cbow_300'
aravec_model = gensim.models.Word2Vec.load(model_path)

In [ ]:
persist_directory = file_path+'/chroma/AraVec/CBOW/'
newAravecDb2 = True
if newAravecDb2:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_aravec_cbow=  Chroma.from_texts(texts=documents, embedding=aravec_embeddings, persist_directory=persist_directory)
  vectorstore_aravec_cbow.persist()
else:
  vectorstore_aravec_cbow = Chroma(persist_directory=persist_directory, embedding_function=aravec_embeddings)
  print(vectorstore_aravec_cbow._collection.count())
retriever_aravec_cbow = vectorstore_aravec_cbow.as_retriever(search_type="similarity", search_kwargs={'k': 5})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  warn_deprecated(


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_aravec_cbow)
printRecallValues(recall_Matrix)
aravec2_MRR = calculate_MRR(recall_Matrix)
print(aravec2_MRR)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


recall @1: 0.23440860215053763
recall @3: 0.3899641577060932
recall @5: 0.4659498207885305
0.23440860215053763 0.3899641577060932 0.4659498207885305
0.3205973715651128


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_aravec_cbow)
printRecallValues(recall_Matrix)
aravec2_MRR2 = calculate_MRR(recall_Matrix)
print(aravec2_MRR2)

recall @1: 0.25519713261648747
recall @3: 0.4136200716845878
recall @5: 0.4931899641577061
0.25519713261648747 0.4136200716845878 0.4931899641577061
0.3440382317801666


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_aravec_cbow)
printRecallValues(recall_Matrix)
aravec2_MRR3 = calculate_MRR(recall_Matrix)
print(aravec2_MRR3)

recall @1: 0.24874551971326164
recall @3: 0.4064516129032258
recall @5: 0.4838709677419355
0.24874551971326164 0.4064516129032258 0.4838709677419355
0.3359378733572275


##AraBert

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
# Load AraBERT model and tokenizer
model_name = "aubmindlab/bert-base-arabertv02"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# Define the embedding function
def arabert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use the mean of the last hidden state as the sentence embedding
    embeddings = outputs.last_hidden_state.mean(dim=1)
    embeddings_np = embeddings.numpy().flatten()

    return embeddings_np.tolist()




# Create the embedding function
class AraBERTEmbeddings:
    def embed_documents(self, texts):
        return [arabert_embedding(text) for text in texts]

    def embed_query(self, text):
        return arabert_embedding(text)

# Use AraBERT embeddings
arabert_embeddings = AraBERTEmbeddings()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/381 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/825k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [ ]:
persist_directory = file_path+'/chroma/AraBert/'
newAraBertDb = False
if newAraBertDb:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_arabert=  Chroma.from_texts(texts=documents, embedding=arabert_embeddings, persist_directory=persist_directory)
  vectorstore_arabert.persist()
  newAraBertDb = False
else:
  vectorstore_arabert = Chroma(persist_directory=persist_directory, embedding_function=arabert_embeddings)
  print(vectorstore_arabert._collection.count())
retriever_arabert = vectorstore_arabert.as_retriever(search_type="similarity", search_kwargs={'k': 5})

460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_arabert)
printRecallValues(recall_Matrix)
arabert_MRR = calculate_MRR(recall_Matrix)
print(arabert_MRR)

recall @1: 0.31756272401433694
recall @3: 0.513978494623656
recall @5: 0.5741935483870968
0.31756272401433694 0.513978494623656 0.5741935483870968
0.41817204301075256


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_arabert)
printRecallValues(recall_Matrix)
arabert_MRR2 = calculate_MRR(recall_Matrix)
print(arabert_MRR2)

recall @1: 0.34767025089605735
recall @3: 0.5620071684587814
recall @5: 0.6315412186379928
0.34767025089605735 0.5620071684587814 0.6315412186379928
0.45853046594982083


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_arabert)
printRecallValues(recall_Matrix)
arabert_MRR3 = calculate_MRR(recall_Matrix)
print(arabert_MRR3)

recall @1: 0.3433691756272401
recall @3: 0.5562724014336917
recall @5: 0.6258064516129033
0.3433691756272401 0.5562724014336917 0.6258064516129033
0.4541218637992833


##BGE

In [ ]:
!pip install -U FlagEmbedding

In [ ]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.7 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer
from FlagEmbedding import BGEM3FlagModel
import torch
#from langchain.vectorstores import Chroma

# Load the tokenizer and model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-m3')
model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=True)

# Function to generate embeddings without batching
def embed_texts(texts):
    embeddings = []
    for text in texts:
        with torch.no_grad():
            output = model.encode([text], return_dense=True)
            embeddings.append(output['dense_vecs'][0].tolist())
    return embeddings

class BGEEmbeddings:
    def embed_documents(self, texts):
        return embed_texts(texts)

    def embed_query(self, text):
        return embed_texts([text])[0]

embedding_bge = BGEEmbeddings()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

...

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

sparse_linear.pt:   0%|          | 0.00/3.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

onnx/tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

In [ ]:
persist_directory = file_path+'/chroma/BGE/'
newBGEDb = False
if newBGEDb:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_bge=  Chroma.from_texts(texts=documents, embedding=embedding_bge, persist_directory=persist_directory)
  vectorstore_bge.persist()
else:
  vectorstore_bge = Chroma(persist_directory=persist_directory, embedding_function=embedding_bge)
  print(vectorstore_bge._collection.count())
retriever_bge = vectorstore_bge.as_retriever(search_type="similarity", search_kwargs={'k': 5})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  warn_deprecated(


460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_bge)
printRecallValues(recall_Matrix)
bge_MRR = calculate_MRR(recall_Matrix)
print(bge_MRR)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


recall @1: 0.6272401433691757
recall @3: 0.8716845878136201
recall @5: 0.9089605734767026
0.6272401433691757 0.8716845878136201 0.9089605734767026
0.7481362007168473


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_bge)
printRecallValues(recall_Matrix)
bge_MRR2 = calculate_MRR(recall_Matrix)
print(bge_MRR2)

recall @1: 0.6437275985663082
recall @3: 0.9096774193548387
recall @5: 0.9433691756272401
0.6437275985663082 0.9096774193548387 0.9433691756272401
0.7735961768219844


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_bge)
printRecallValues(recall_Matrix)
bge_MRR3 = calculate_MRR(recall_Matrix)
print(bge_MRR3)

recall @1: 0.6236559139784946
recall @3: 0.8853046594982079
recall @5: 0.9218637992831541
0.6236559139784946 0.8853046594982079 0.9218637992831541
0.7523297491039439


## E5 Large (1024)

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

# Function to generate embeddings
def embed_texts(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().tolist())
    return embeddings


tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/690 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

In [ ]:
model

XLMRobertaModel(
  (embeddings): XLMRobertaEmbeddings(
    (word_embeddings): Embedding(250002, 1024, padding_idx=1)
    (position_embeddings): Embedding(514, 1024, padding_idx=1)
    (token_type_embeddings): Embedding(1, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): XLMRobertaEncoder(
    (layer): ModuleList(
      (0-23): 24 x XLMRobertaLayer(
        (attention): XLMRobertaAttention(
          (self): XLMRobertaSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): XLMRobertaSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwi

In [ ]:
class E5Embeddings:
    def embed_documents(self, texts):
        return embed_texts(texts)

    def embed_query(self, text):
        return embed_texts([text])[0]

embedding_E5L = E5Embeddings()

In [ ]:
persist_directory = file_path+'/chroma/E5_large/'
newE5L= False
if newE5L:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_e5l=  Chroma.from_texts(texts=documents, embedding=embedding_E5L, persist_directory=persist_directory)
  vectorstore_e5l.persist()
else:
  vectorstore_e5l = Chroma(persist_directory=persist_directory, embedding_function=embedding_E5L)
  print(vectorstore_e5l._collection.count())
retriever_e5l = vectorstore_e5l.as_retriever(search_type="similarity", search_kwargs={'k': 5})

460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_e5l)
printRecallValues(recall_Matrix)
e5l_MRR = calculate_MRR(recall_Matrix)
print(e5l_MRR)


recall @1: 0.6860215053763441
recall @3: 0.8960573476702509
recall @5: 0.9268817204301075
0.6860215053763441 0.8960573476702509 0.9268817204301075
0.7897610513739556


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_e5l)
printRecallValues(recall_Matrix)
e5l_MRR2 = calculate_MRR(recall_Matrix)
print(e5l_MRR2)



recall @1: 0.7189964157706094
recall @3: 0.9376344086021505
recall @5: 0.9627240143369176
0.7189964157706094 0.9376344086021505 0.9627240143369176
0.8255555555555565


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_e5l)
printRecallValues(recall_Matrix)
e5l_MRR3 = calculate_MRR(recall_Matrix)
print(e5l_MRR3)

## E5 Small

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

# Function to generate embeddings
def embed_texts(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().tolist())
    return embeddings


class E5EmbeddingsSmall:
    def embed_documents(self, texts):
        return embed_texts(texts)

    def embed_query(self, text):
        return embed_texts([text])[0]

embedding_E5_small = E5EmbeddingsSmall()

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/655 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

In [ ]:
device

device(type='cuda')

In [ ]:
persist_directory = file_path+'/chroma/E5_small/'
newE5S= False
if newE5S:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_e5s=  Chroma.from_texts(texts=documents, embedding=embedding_E5_small, persist_directory=persist_directory)
  vectorstore_e5s.persist()
else:
  vectorstore_e5s = Chroma(persist_directory=persist_directory, embedding_function=embedding_E5_small)
  print(vectorstore_e5s._collection.count())
retriever_e5s = vectorstore_e5s.as_retriever(search_type="similarity", search_kwargs={'k': 5})

460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_e5s)
printRecallValues(recall_Matrix)
e5_s_MRR = calculate_MRR(recall_Matrix)
print(e5_s_MRR)

recall @1: 0.6315412186379928
recall @3: 0.860931899641577
recall @5: 0.8931899641577061
0.6315412186379928 0.860931899641577 0.8931899641577061
0.7424014336917575


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_e5s)
printRecallValues(recall_Matrix)
e5_s_MRR2 = calculate_MRR(recall_Matrix)
print(e5_s_MRR2)

recall @1: 0.6645161290322581
recall @3: 0.9032258064516129
recall @5: 0.9340501792114695
0.6645161290322581 0.9032258064516129 0.9340501792114695
0.7807048984468353


In [ ]:
recall_Matrix = calcRecValues3(df, 5, retriever_e5s)
printRecallValues(recall_Matrix)
e5_s_MRR3 = calculate_MRR(recall_Matrix)
print(e5_s_MRR3)

recall @1: 0.6315412186379928
recall @3: 0.875268817204301
recall @5: 0.9154121863799283
0.6315412186379928 0.875268817204301 0.9154121863799283
0.7520310633213874


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_e5s)
printRecallValues(recall_Matrix)

recall @1: 0.6351254480286739
recall @3: 0.8788530465949821
recall @5: 0.9175627240143369
0.6351254480286739 0.8788530465949821 0.9175627240143369


In [ ]:
recall_Matrix = calcRecValues2(df, 5, retriever_e5s)
printRecallValues(recall_Matrix)

recall @1: 0.6637992831541218
recall @3: 0.9046594982078853
recall @5: 0.9369175627240144
0.6637992831541218 0.9046594982078853 0.9369175627240144


In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-base')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#model.to(device)

# Function to generate embeddings
def embed_texts(texts):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings.append(outputs.last_hidden_state.mean(dim=1).squeeze().tolist())
    return embeddings


class E5EmbeddingsBase:
    def embed_documents(self, texts):
        return embed_texts(texts)

    def embed_query(self, text):
        return embed_texts([text])[0]

embedding_E5_base = E5EmbeddingsBase()

tokenizer_config.json:   0%|          | 0.00/418 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
persist_directory = file_path+'/chroma/E5_base/'
newE5B= False
if newE5B:
  !rm -rf persist_directory # remove old database files if any
  vectorstore_e5b=  Chroma.from_texts(texts=documents, embedding=embedding_E5_base, persist_directory=persist_directory)
  vectorstore_e5b.persist()
else:
  vectorstore_e5b = Chroma(persist_directory=persist_directory, embedding_function=embedding_E5_base)
  print(vectorstore_e5b._collection.count())
retriever_e5b = vectorstore_e5b.as_retriever(search_type="similarity", search_kwargs={'k': 5})

460


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_e5b)
printRecallValues(recall_Matrix)

recall @1: 0.403584229390681
recall @3: 0.6408602150537634
recall @5: 0.6939068100358423
0.403584229390681 0.6408602150537634 0.6939068100358423


In [ ]:
recall_Matrix = calcRecValues(df, 5, retriever_e5b)
printRecallValues(recall_Matrix)

recall @1: 0.4057347670250896
recall @3: 0.6523297491039427
recall @5: 0.7060931899641577
0.4057347670250896 0.6523297491039427 0.7060931899641577


In [ ]:
device

NameError: name 'device' is not defined

In [ ]:
!pip show chromadb


Name: chromadb
Version: 0.5.5
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, tenacity, tokenizers, tqdm, typer, typing-extensions, uvicorn
Required-by: langchain-chroma
